In [1]:
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModelForTokenClassification
from pathlib import Path
import numpy as np
import torch
from tokenizers import AddedToken
from tqdm.notebook import tqdm
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import pandas as pd
from seqeval.metrics import recall_score, precision_score, f1_score, accuracy_score
from datasets import Dataset

kaggle=False

path="/kaggle/input/pii-detection-removal-from-educational-data" if kaggle else "data"
train_path = path + "/train.json"
test_path = path + "/test.json"

model_path = "/kaggle/input/huggingfacedebertav3variants/deberta-v3-base" if kaggle else "microsoft/deberta-v3-base"

if not kaggle: import neptune

In [2]:
cross_entropy_weight_multi = 400

CROSS_ENTROPY_WEIGHTS = [cross_entropy_weight_multi]*12
CROSS_ENTROPY_WEIGHTS.append(1)

parameter= {
    "model": model_path,
    "max_length": 1024,
    "inference_max_length": 2000,
    "batch_size": 4,
    "inference_batch_size": 1,
    "lr": 5e-05,
    "lr_scale_unfreeze": 0.1,
    "filter_no_pii_percent_allow": 0.2,
    "notebook": "20_deberta base_1024len.ipynb",
    "CROSS_ENTROPY_WEIGHT_MULTI": cross_entropy_weight_multi,
    "epochs_before_unfreeze": 1,
    "epochs_after_unfreeze": 6,
    "repeat_unfreeze_train_n_times": 2,
    "validate_every_n_epochs": 2,
    "train_test_split": 0.2,
    "num_proc": 16, 
    "freeze_embeddings": False,
    "freeze_layers": 6
}

print(parameter)

{'model': 'microsoft/deberta-v3-base', 'max_length': 1024, 'inference_max_length': 2000, 'batch_size': 4, 'inference_batch_size': 1, 'lr': 5e-05, 'lr_scale_unfreeze': 0.1, 'filter_no_pii_percent_allow': 0.2, 'notebook': '20_deberta base_1024len.ipynb', 'CROSS_ENTROPY_WEIGHT_MULTI': 400, 'epochs_before_unfreeze': 1, 'epochs_after_unfreeze': 6, 'repeat_unfreeze_train_n_times': 2, 'validate_every_n_epochs': 2, 'train_test_split': 0.2, 'num_proc': 16, 'freeze_embeddings': False, 'freeze_layers': 6}


In [3]:
target = [
    'B-EMAIL', 'B-ID_NUM', 'B-NAME_STUDENT', 'B-PHONE_NUM', 
    'B-STREET_ADDRESS', 'B-URL_PERSONAL', 'B-USERNAME', 'I-ID_NUM', 
    'I-NAME_STUDENT', 'I-PHONE_NUM', 'I-STREET_ADDRESS', 'I-URL_PERSONAL'
]

In [4]:
from itertools import chain
import json

data = json.load(open(train_path))
all_labels = sorted(list(set(chain(*[x["labels"] for x in data]))))
label2id = {l: i for i,l in enumerate(all_labels)}
id2label = {v:k for k,v in label2id.items()}

In [5]:
import random

def tokenize(example, tokenizer, label2id, max_length, all_labels_list):
    text = []
    import numpy as np

    # these are at the character level
    labels = []
    targets = []

    for t, l, ws in zip(example["tokens"], example["labels"], example["trailing_whitespace"]):

        text.append(t)
        labels.extend([l]*len(t))
        
        if l in all_labels_list:
            targets.append(1)
        else:
            targets.append(0)
        # if there is trailing whitespace
        if ws:
            text.append(" ")
            labels.append("O")

    tokenized = tokenizer("".join(text), return_offsets_mapping=True, truncation=True, max_length=max_length, padding="max_length")
    
    target_num = sum(targets)
    labels = np.array(labels)

    text = "".join(text)
    token_labels = []

    for start_idx, end_idx in tokenized.offset_mapping:

        # CLS token
        if start_idx == 0 and end_idx == 0: 
            token_labels.append(label2id["O"])
            continue

        # case when token starts with whitespace
        if text[start_idx].isspace():
            start_idx += 1

        try:
            token_labels.append(label2id[labels[start_idx]])
        except:
            token_labels.append(label2id["O"])

    length = len(tokenized.input_ids)

    return {
        **tokenized,
        "labels": token_labels,
        "length": length,
        "target_num": target_num,
        "group": 1 if target_num>0 else 0
    }

# https://www.kaggle.com/competitions/pii-detection-removal-from-educational-data/discussion/468844
def filter_no_pii(example, percent_allow=parameter["filter_no_pii_percent_allow"]):
    # Return True if there is PII
    # Or 20% of the time if there isn't
    has_pii = set("O") != set(example["labels"])
    return has_pii or (random.random() < percent_allow)

In [6]:
data = json.load(open(train_path))
ds = Dataset.from_dict({
    "full_text": [x["full_text"] for x in data],
    "document": [str(x["document"]) for x in data],
    "tokens": [x["tokens"] for x in data],
    "trailing_whitespace": [x["trailing_whitespace"] for x in data],
    "labels": [x["labels"] for x in data],
})
    
tokenizer = AutoTokenizer.from_pretrained(parameter["model"])
ds = ds.map(tokenize, fn_kwargs={"tokenizer": tokenizer, "label2id": label2id, "max_length": parameter["max_length"], "all_labels_list": target}, num_proc=parameter["num_proc"])
ds=ds.filter(filter_no_pii, num_proc=parameter["num_proc"])


data_len=len(ds)
train_len=int(len(ds)*(1-parameter["train_test_split"]))
valid_len=len(ds)-train_len
train_data_idx=np.random.choice(data_len, train_len, replace=False)
valid_data_idx=np.array(list(set(range(data_len))-set(train_data_idx)))
print("train_len", train_len)
print("valid_len", valid_len)

# split ds in train and valid
train_ds=ds.select(train_data_idx)
valid_ds=ds.select(valid_data_idx)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
c:\Users\Bernd\anaconda3\envs\mytorch\lib\site-packages\transformers\convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map (num_proc=16):   0%|          | 0/6807 [00:00<?, ? examples/s]

Filter (num_proc=16):   0%|          | 0/6807 [00:00<?, ? examples/s]

train_len 5445
valid_len 1362


In [7]:
train_ds[0]["input_ids"]

[1,
 6348,
 6190,
 377,
 2169,
 1291,
 270,
 3513,
 45730,
 377,
 53779,
 1637,
 6738,
 429,
 14700,
 4305,
 277,
 312,
 517,
 261,
 53779,
 303,
 1666,
 351,
 553,
 262,
 11412,
 866,
 265,
 266,
 483,
 264,
 355,
 2282,
 277,
 262,
 23114,
 465,
 265,
 308,
 374,
 260,
 463,
 465,
 265,
 312,
 5311,
 261,
 273,
 3232,
 275,
 266,
 509,
 265,
 467,
 6998,
 264,
 10151,
 262,
 568,
 272,
 413,
 465,
 265,
 260,
 589,
 386,
 5658,
 261,
 273,
 286,
 757,
 1568,
 3498,
 264,
 398,
 262,
 610,
 1355,
 265,
 262,
 374,
 306,
 333,
 267,
 266,
 406,
 264,
 406,
 1599,
 260,
 329,
 3498,
 303,
 1379,
 264,
 8509,
 292,
 347,
 1568,
 1712,
 262,
 713,
 272,
 308,
 1016,
 702,
 276,
 297,
 1329,
 264,
 286,
 1239,
 277,
 262,
 568,
 306,
 374,
 277,
 260,
 463,
 2996,
 293,
 6601,
 77962,
 53144,
 261,
 3234,
 4735,
 1872,
 288,
 7583,
 261,
 277,
 262,
 567,
 542,
 265,
 262,
 586,
 261,
 53779,
 335,
 619,
 4337,
 1530,
 264,
 676,
 1138,
 292,
 339,
 284,
 4114,
 266,
 735,
 260,
 3089,
 26

In [8]:
def tokenize_inference(example, tokenizer, max_length):
        text = []
        for t,  ws in zip(example["tokens"], example["trailing_whitespace"]):
            text.append(t)
            if ws:
                text.append(" ")
        tokenized = tokenizer("".join(text), return_offsets_mapping=True, truncation=True, max_length=max_length, padding="max_length")
        text = "".join(text)
        length = len(tokenized.input_ids)
        return {
            **tokenized,
            "length": length,
        }
        
class TestTokenizer():
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
    
    def preprocess(self, example):
        # Preprocess the tokens and labels by adding trailing whitespace and labels
        tokens = []
        tokens_without_ws = []
        token_map = [] # Use the index as labels
        index = 0
        for token, t_ws in zip(example["tokens"], example["trailing_whitespace"]):
            tokens_without_ws.append(token)
            tokens.append(token)
            token_map.extend([index] * len(token))
            # Added trailing whitespace and label if true and 
            if t_ws:
                tokens.append(" ")
                token_map.append(-1)
            index += 1
        return tokens, token_map, tokens_without_ws
    
    def tokenize(self, example):
        tokens, token_map, tokens_without_ws = self.preprocess(example)
        text = "".join(tokens)
        tokenized = self.tokenizer(text, return_offsets_mapping=True, padding="max_length",
                                   truncation=True, max_length=parameter["inference_max_length"])
        return {**tokenized, "token_map": token_map, "tokens": tokens, "tokens_without_ws": tokens_without_ws} 

class PiiDatasetInference(torch.utils.data.Dataset):
        def __init__(self, dataset, tokenizer):
            self.dataset = dataset
            self.tokenizer=TestTokenizer(tokenizer)
            
        def __getitem__(self, idx):
            vals=self.tokenizer.tokenize(self.dataset[idx])
            input_ids = torch.tensor(vals["input_ids"])
            attention_mask = torch.tensor(vals["attention_mask"])
            document_id = self.dataset[idx]["document"]
            return input_ids, attention_mask, document_id, vals
        
        def __len__(self):
            return len(self.dataset)

# Convert preds to a list of dictionaries
def to_test_submission(preds=None, dataset=None, document_ids=None, id2label=None):
    triplets = []
    row_id = 0
    results = []
    
    for i in range(len(preds)):
        input_ids, attention_mask, document_id, vals = dataset[i]
        token_map=vals["token_map"]
        offsets=vals["offset_mapping"]
        tokens=vals["tokens_without_ws"]
        #print("tokens", tokens)
        pred=preds[i]
        original_text = tokenizer.decode(input_ids)[6:] # skip CLS token
        #print("original_text", original_text)
        #print("token_map", token_map)
        #print("offsets", offsets)   
        #print("pred", pred)

        for token_pred, input_id, (start_idx, end_idx) in zip(pred, input_ids, offsets):
            #print("\nnow doing ", start_idx,  end_idx, token_pred)
            if start_idx == 0 and end_idx == 0: # Skip 0 offset
                continue
            # Skip spaces 
            while start_idx < len(token_map):
                #print("loop, start_idx now", start_idx) 
                #print(" tokens[token_map[start_idx]]: ", tokens[token_map[start_idx]] if not tokens[token_map[start_idx]].isspace() else "WHITESPACE")          
                if token_map[start_idx] == -1: # Skip unknown tokens               
                    start_idx += 1
                elif tokens[token_map[start_idx]].isspace(): # Skip white space
                    start_idx += 1
                else:
                    break
            # Ensure start index < length
            if start_idx < len(token_map):
                token_id = token_map[start_idx]
                #print("token_id", token_id)
                #token_id= input_id.item()
                label_pred = id2label[token_pred.item()]
                #print("label_pred", label_pred)
                # ignore "O" and whitespace preds
                if label_pred != "O" and token_id != -1:
                    #print("is PII", token_id, label_pred)
                    token_str = tokens[token_id]
                    triplet = (label_pred, token_id, token_str)
                    if triplet not in triplets:
                        results.append({
                            "row_id": row_id, 
                            "document": document_id,
                            "token": token_id, 
                            "label": label_pred,
                            "token_str": token_str
                        })
                        triplets.append(triplet)
                        row_id += 1

    # Create a dataframe 
    return results

def create_submission(model, filename="submission.csv"):
    data = json.load(open(train_path))
    from itertools import chain
    all_labels = sorted(list(set(chain(*[x["labels"] for x in data]))))
    label2id = {l: i for i,l in enumerate(all_labels)}
    id2label = {v:k for k,v in label2id.items()}

    data=json.load(open(test_path))
    tokenizer = AutoTokenizer.from_pretrained(parameter["model"])
    my_dataset=PiiDatasetInference(data, tokenizer)
    loader=torch.utils.data.DataLoader(my_dataset, batch_size=1, shuffle=False)

    device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    
    # stack all predictions into tensor
    all_preds = []

    for id, attention_mask, document_ids, vals in loader:
        id=id.to(device)
        attention_mask=attention_mask.to(device)
        preds=model(id, attention_mask).get('logits').argmax(dim=2)
        all_preds.append(preds)
        #for pred, id in zip(preds.flatten(), id.flatten()):
        #    if pred != 12:
                #print(f"Document: {document_id.item()} TOKEN:{tokenizer.decode(id)}  --- pred:{id2label[pred.item()]}")
        #        output[row_id]={"document":document_id.item(), "token":id.item(), "label":id2label[pred.item()]}
        #        row_id+=1
        #for pred, id in zip(preds.flatten(), id.flatten()):
        #    if pred != 12:
        #        print(f"TOKEN:{tokenizer.decode(id)}  --- pred:{id2label[pred.item()]}")
    
   
    all_preds = torch.cat(all_preds, dim=0)
    
    results = to_test_submission(preds=all_preds, dataset=my_dataset, document_ids=document_ids, id2label=id2label)
    if len(results) == 0:
        print("Error in create_submission(): No predictions made, probably because the model is not learning. Check the model and the data.")
        return
    df = pd.DataFrame(results)
    df=df[["row_id", "document", "token", "label"]]
    print(df)
    df.to_csv(filename, index=False)

#create_submission(MyModel(parameter['model'], len(label2id)).to(device), "submission_just_dumb.csv")
# create_submission(model, "submission.csv")


In [9]:
from transformers import DataCollatorForTokenClassification

collator = DataCollatorForTokenClassification(tokenizer, pad_to_multiple_of=16)

In [10]:
# using Trainer and TrainingArguments from transformers


def compute_metrics(p, all_labels):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [all_labels[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [all_labels[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    
    recall = recall_score(true_labels, true_predictions)
    precision = precision_score(true_labels, true_predictions)
    f1_score = (1 + 5*5) * recall * precision / (5*5*precision + recall)
    
    results = {
        'recall': recall,
        'precision': precision,
        'f1': f1_score
    }
    return results

if not kaggle:
    from transformers.integrations import NeptuneCallback

    run = neptune.init_run(
        project="bernd.heidemann/PII",
        api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIzNjBlYzVkNi0zZTUwLTQ1ODYtODhlNC02NDUxNDg0MDdjNzUifQ==",
    )  # your credentials
    run["parameters"] = {
    **parameter
    }

    neptune_callback = NeptuneCallback(run=run, log_model_weights=False, log_parameters=False)

from functools import partial

def get_trainer(model, train_dataloader, valid_dataloader):
    training_args = TrainingArguments(
        output_dir='./results',          # output directory
        num_train_epochs=parameter["epochs_before_unfreeze"],              # total number of training epochs
        per_device_train_batch_size=parameter["batch_size"],  # batch size per device during training
        per_device_eval_batch_size=parameter["inference_batch_size"],   # batch size for evaluation
        warmup_steps=500,                # number of warmup steps for learning rate scheduler
        weight_decay=0.01,               # strength of weight decay
        logging_dir='./logs',            # directory for storing logs
        logging_steps=10,
        evaluation_strategy="steps",
        eval_steps=400,
        save_steps=400,
        save_total_limit=3,
        load_best_model_at_end=True,
        metric_for_best_model="f1" if not kaggle else "eval_loss",
        greater_is_better=True,
        overwrite_output_dir=True,
        report_to="none",
        learning_rate=parameter["lr"],

        #callback for neptune
    )

    class MyTrainer(Trainer):
        def __init__(self, model=None, args=None, train_dataset=None, eval_dataset=None, compute_metrics=None, callbacks=None):
            super().__init__(model=model, args=args, train_dataset=train_dataset, eval_dataset=eval_dataset, compute_metrics=compute_metrics, callbacks=callbacks)
            # Definieren Sie hier Ihre Gewichte für die Klassen, z.B. torch.tensor([1.0, 2.0, 0.5])
            self.weight = torch.tensor(CROSS_ENTROPY_WEIGHTS).to(device)
            self.loss_func=torch.nn.CrossEntropyLoss(ignore_index=-100, weight=torch.tensor(CROSS_ENTROPY_WEIGHTS, dtype=torch.float32).to(device))

        def compute_loss(self, model, inputs, return_outputs=False):
            labels = inputs.get("labels")
            outputs = model(**inputs)
            logits = outputs.get('logits')
            loss = self.loss_func(logits.view(-1, self.model.config.num_labels), labels.view(-1))
            return (loss, outputs) if return_outputs else loss

    trainer = MyTrainer(
        model=model,                         # the instantiated 🤗 Transformers model to be trained
        args=training_args,                  # training arguments, defined above
        train_dataset=train_dataloader,         # training dataset
        eval_dataset=valid_dataloader,             # evaluation dataset
        compute_metrics=partial(compute_metrics, all_labels=all_labels) if not kaggle else None,
        callbacks=[neptune_callback] if not kaggle else None
    )
    return trainer

c:\Users\Bernd\anaconda3\envs\mytorch\lib\site-packages\neptune\common\warnings.py:71: NeptuneWarning: The following monitoring options are disabled by default in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', and 'capture_hardware_metrics'. To enable them, set each parameter to 'True' when initializing the run. The monitoring will continue until you call run.stop() or the kernel stops. Also note: Your source files can only be tracked if you pass the path(s) to the 'source_code' argument. For help, see the Neptune docs: https://docs.neptune.ai/logging/source_code/
  warnings.warn(


https://app.neptune.ai/bernd.heidemann/PII/e/PII-216


In [11]:
def unfreeze(model):
    for param in model.base_model.parameters():
        param.requires_grad = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
# set environment variables: TOKENIZERS_PARALLELISM=false
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

tokenizer = AutoTokenizer.from_pretrained(parameter["model"])
tokenizer.add_tokens(AddedToken("\n", normalized=False))

model = AutoModelForTokenClassification.from_pretrained(
    parameter["model"],
    num_labels=len(all_labels),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

if parameter['freeze_embeddings']:
    for param in model.deberta.embeddings.parameters():
        param.requires_grad = False
        
if parameter['freeze_layers'] > 0:
    for layer in model.deberta.encoder.layer[:parameter['freeze_layers']]:
        for param in layer.parameters():
            param.requires_grad = False

print(model.config)
#my_model=MyModel(parameter['model'], len(label2id))

trainer=get_trainer(model, train_ds, valid_ds)
trainer.train()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DebertaV2Config {
  "_name_or_path": "microsoft/deberta-v3-base",
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "B-EMAIL",
    "1": "B-ID_NUM",
    "2": "B-NAME_STUDENT",
    "3": "B-PHONE_NUM",
    "4": "B-STREET_ADDRESS",
    "5": "B-URL_PERSONAL",
    "6": "B-USERNAME",
    "7": "I-ID_NUM",
    "8": "I-NAME_STUDENT",
    "9": "I-PHONE_NUM",
    "10": "I-STREET_ADDRESS",
    "11": "I-URL_PERSONAL",
    "12": "O"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-EMAIL": 0,
    "B-ID_NUM": 1,
    "B-NAME_STUDENT": 2,
    "B-PHONE_NUM": 3,
    "B-STREET_ADDRESS": 4,
    "B-URL_PERSONAL": 5,
    "B-USERNAME": 6,
    "I-ID_NUM": 7,
    "I-NAME_STUDENT": 8,
    "I-PHONE_NUM": 9,
    "I-STREET_ADDRESS": 10,
    "I-URL_PERSONAL": 11,
    "O": 12
  },
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deber

c:\Users\Bernd\anaconda3\envs\mytorch\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1362 [00:00<?, ?it/s]

{'loss': 2.9192, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.01}
{'loss': 2.7746, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.01}
{'loss': 2.5098, 'learning_rate': 3e-06, 'epoch': 0.02}
{'loss': 2.2666, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.03}
{'loss': 1.8006, 'learning_rate': 5e-06, 'epoch': 0.04}
{'loss': 1.6844, 'learning_rate': 6e-06, 'epoch': 0.04}
{'loss': 1.1798, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.05}
{'loss': 0.8757, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.06}
{'loss': 0.499, 'learning_rate': 9e-06, 'epoch': 0.07}
{'loss': 1.0246, 'learning_rate': 1e-05, 'epoch': 0.07}
{'loss': 0.7192, 'learning_rate': 1.1000000000000001e-05, 'epoch': 0.08}
{'loss': 0.8068, 'learning_rate': 1.2e-05, 'epoch': 0.09}
{'loss': 1.1109, 'learning_rate': 1.3000000000000001e-05, 'epoch': 0.1}
{'loss': 0.4495, 'learning_rate': 1.4000000000000001e-05, 'epoch': 0.1}
{'loss': 0.306, 'learning_rate': 1.5e-05, 'epoch': 0.11}
{'loss': 0.3918, 'learning_

...

{'eval_loss': 0.09517715871334076, 'eval_recall': 0.19258202567760344, 'eval_precision': 0.09527170077628794, 'eval_f1': 0.18530250237567314, 'eval_runtime': 45.9915, 'eval_samples_per_second': 29.614, 'eval_steps_per_second': 29.614, 'epoch': 0.15}
{'loss': 0.0718, 'learning_rate': 2.1e-05, 'epoch': 0.15}
{'loss': 0.3089, 'learning_rate': 2.2000000000000003e-05, 'epoch': 0.16}
{'loss': 0.0741, 'learning_rate': 2.3000000000000003e-05, 'epoch': 0.17}
{'loss': 0.6722, 'learning_rate': 2.4e-05, 'epoch': 0.18}
{'loss': 0.2051, 'learning_rate': 2.5e-05, 'epoch': 0.18}
{'loss': 0.3092, 'learning_rate': 2.6000000000000002e-05, 'epoch': 0.19}
{'loss': 0.0083, 'learning_rate': 2.7000000000000002e-05, 'epoch': 0.2}
{'loss': 0.317, 'learning_rate': 2.8000000000000003e-05, 'epoch': 0.21}
{'loss': 0.0433, 'learning_rate': 2.9e-05, 'epoch': 0.21}
{'loss': 0.2264, 'learning_rate': 3e-05, 'epoch': 0.22}
{'loss': 0.0336, 'learning_rate': 3.1e-05, 'epoch': 0.23}
{'loss': 0.0495, 'learning_rate': 3.20000

  0%|          | 0/1362 [00:00<?, ?it/s]

{'eval_loss': 0.0422653965651989, 'eval_recall': 0.456490727532097, 'eval_precision': 0.4657933042212518, 'eval_f1': 0.4568416428728311, 'eval_runtime': 43.1015, 'eval_samples_per_second': 31.6, 'eval_steps_per_second': 31.6, 'epoch': 0.29}
{'loss': 0.1514, 'learning_rate': 4.1e-05, 'epoch': 0.3}
{'loss': 0.0053, 'learning_rate': 4.2e-05, 'epoch': 0.31}
{'loss': 0.1882, 'learning_rate': 4.3e-05, 'epoch': 0.32}
{'loss': 0.0016, 'learning_rate': 4.4000000000000006e-05, 'epoch': 0.32}
{'loss': 0.0993, 'learning_rate': 4.5e-05, 'epoch': 0.33}
{'loss': 0.0108, 'learning_rate': 4.600000000000001e-05, 'epoch': 0.34}
{'loss': 0.0802, 'learning_rate': 4.7e-05, 'epoch': 0.35}
{'loss': 0.0033, 'learning_rate': 4.8e-05, 'epoch': 0.35}
{'loss': 0.0003, 'learning_rate': 4.9e-05, 'epoch': 0.36}
{'loss': 0.11, 'learning_rate': 5e-05, 'epoch': 0.37}
{'loss': 0.0152, 'learning_rate': 4.9419953596287704e-05, 'epoch': 0.37}
{'loss': 0.0311, 'learning_rate': 4.8839907192575406e-05, 'epoch': 0.38}
{'loss': 

  0%|          | 0/1362 [00:00<?, ?it/s]

{'eval_loss': 0.1080680564045906, 'eval_recall': 0.4108416547788873, 'eval_precision': 0.6220302375809935, 'eval_f1': 0.41627751834556376, 'eval_runtime': 43.2732, 'eval_samples_per_second': 31.474, 'eval_steps_per_second': 31.474, 'epoch': 0.44}
{'loss': 0.0005, 'learning_rate': 4.3619489559164736e-05, 'epoch': 0.45}
{'loss': 0.0031, 'learning_rate': 4.303944315545244e-05, 'epoch': 0.46}
{'loss': 0.9597, 'learning_rate': 4.2459396751740146e-05, 'epoch': 0.46}
{'loss': 0.0088, 'learning_rate': 4.187935034802785e-05, 'epoch': 0.47}
{'loss': 0.0895, 'learning_rate': 4.129930394431555e-05, 'epoch': 0.48}
{'loss': 0.0018, 'learning_rate': 4.071925754060325e-05, 'epoch': 0.48}
{'loss': 0.0055, 'learning_rate': 4.013921113689095e-05, 'epoch': 0.49}
{'loss': 0.1362, 'learning_rate': 3.9559164733178655e-05, 'epoch': 0.5}
{'loss': 0.1732, 'learning_rate': 3.897911832946636e-05, 'epoch': 0.51}
{'loss': 0.357, 'learning_rate': 3.839907192575406e-05, 'epoch': 0.51}
{'loss': 0.0279, 'learning_rate'

In [ ]:
unfreeze(model)
parameter["lr"] = parameter["lr"] * parameter["lr_scale_unfreeze"]
trainer=get_trainer(model, train_ds, valid_ds)
trainer.train()


c:\Users\Bernd\anaconda3\envs\mytorch\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/5448 [00:00<?, ?it/s]

{'loss': 0.0006, 'learning_rate': 1.0000000000000001e-07, 'epoch': 0.01}
{'loss': 0.0121, 'learning_rate': 2.0000000000000002e-07, 'epoch': 0.01}
{'loss': 0.0008, 'learning_rate': 3.0000000000000004e-07, 'epoch': 0.02}
{'loss': 0.0006, 'learning_rate': 4.0000000000000003e-07, 'epoch': 0.03}
{'loss': 0.0334, 'learning_rate': 5.000000000000001e-07, 'epoch': 0.04}
{'loss': 0.1075, 'learning_rate': 6.000000000000001e-07, 'epoch': 0.04}
{'loss': 0.0009, 'learning_rate': 7.000000000000001e-07, 'epoch': 0.05}
{'loss': 0.0006, 'learning_rate': 8.000000000000001e-07, 'epoch': 0.06}
{'loss': 0.0008, 'learning_rate': 9.000000000000001e-07, 'epoch': 0.07}
{'loss': 0.0014, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.07}
{'loss': 0.0007, 'learning_rate': 1.1e-06, 'epoch': 0.08}
{'loss': 0.0025, 'learning_rate': 1.2000000000000002e-06, 'epoch': 0.09}
{'loss': 0.0017, 'learning_rate': 1.3e-06, 'epoch': 0.1}
{'loss': 0.0014, 'learning_rate': 1.4000000000000001e-06, 'epoch': 0.1}
{'loss': 0.0004

  0%|          | 0/1362 [00:00<?, ?it/s]

{'eval_loss': 0.01020288746803999, 'eval_recall': 0.9641319942611191, 'eval_precision': 0.5808124459809854, 'eval_f1': 0.9402647723603486, 'eval_runtime': 44.5961, 'eval_samples_per_second': 30.541, 'eval_steps_per_second': 30.541, 'epoch': 0.15}
{'loss': 0.0038, 'learning_rate': 2.1000000000000002e-06, 'epoch': 0.15}
{'loss': 0.0007, 'learning_rate': 2.2e-06, 'epoch': 0.16}
{'loss': 0.0002, 'learning_rate': 2.3000000000000004e-06, 'epoch': 0.17}
{'loss': 0.0001, 'learning_rate': 2.4000000000000003e-06, 'epoch': 0.18}
{'loss': 0.0003, 'learning_rate': 2.5e-06, 'epoch': 0.18}
{'loss': 0.149, 'learning_rate': 2.6e-06, 'epoch': 0.19}
{'loss': 0.0003, 'learning_rate': 2.7000000000000004e-06, 'epoch': 0.2}
{'loss': 0.0023, 'learning_rate': 2.8000000000000003e-06, 'epoch': 0.21}
{'loss': 0.0001, 'learning_rate': 2.9e-06, 'epoch': 0.21}
{'loss': 0.1787, 'learning_rate': 3e-06, 'epoch': 0.22}
{'loss': 0.0001, 'learning_rate': 3.1000000000000004e-06, 'epoch': 0.23}
{'loss': 0.0032, 'learning_ra

...

  0%|          | 0/1362 [00:00<?, ?it/s]

{'eval_loss': 0.010243733413517475, 'eval_recall': 0.9182209469153515, 'eval_precision': 0.6280667320902846, 'eval_f1': 0.902190414226849, 'eval_runtime': 50.3562, 'eval_samples_per_second': 27.047, 'eval_steps_per_second': 27.047, 'epoch': 1.03}
{'loss': 0.0001, 'learning_rate': 4.080436540016168e-06, 'epoch': 1.04}
{'loss': 0.0011, 'learning_rate': 4.0703314470493135e-06, 'epoch': 1.04}
{'loss': 0.0872, 'learning_rate': 4.060226354082458e-06, 'epoch': 1.05}
{'loss': 0.0013, 'learning_rate': 4.0501212611156026e-06, 'epoch': 1.06}
{'loss': 0.0016, 'learning_rate': 4.040016168148747e-06, 'epoch': 1.06}
{'loss': 0.0001, 'learning_rate': 4.0299110751818925e-06, 'epoch': 1.07}
{'loss': 0.0007, 'learning_rate': 4.019805982215036e-06, 'epoch': 1.08}
{'loss': 0.0017, 'learning_rate': 4.0097008892481816e-06, 'epoch': 1.09}
{'loss': 0.0063, 'learning_rate': 3.999595796281326e-06, 'epoch': 1.09}
{'loss': 0.0003, 'learning_rate': 3.989490703314471e-06, 'epoch': 1.1}
{'loss': 0.0001, 'learning_rat

  0%|          | 0/1362 [00:00<?, ?it/s]

{'eval_loss': 0.008026955649256706, 'eval_recall': 0.93974175035868, 'eval_precision': 0.5987202925045704, 'eval_f1': 0.9195960905016469, 'eval_runtime': 47.0298, 'eval_samples_per_second': 28.96, 'eval_steps_per_second': 28.96, 'epoch': 1.17}
{'loss': 0.0003, 'learning_rate': 3.878334680679063e-06, 'epoch': 1.18}
{'loss': 0.0006, 'learning_rate': 3.868229587712207e-06, 'epoch': 1.19}
{'loss': 0.0017, 'learning_rate': 3.858124494745352e-06, 'epoch': 1.2}
{'loss': 0.0008, 'learning_rate': 3.848019401778497e-06, 'epoch': 1.2}
{'loss': 0.0003, 'learning_rate': 3.837914308811641e-06, 'epoch': 1.21}
{'loss': 0.003, 'learning_rate': 3.827809215844786e-06, 'epoch': 1.22}
{'loss': 0.0009, 'learning_rate': 3.817704122877931e-06, 'epoch': 1.23}
{'loss': 0.0, 'learning_rate': 3.8075990299110754e-06, 'epoch': 1.23}
{'loss': 0.0004, 'learning_rate': 3.7974939369442203e-06, 'epoch': 1.24}
{'loss': 0.0004, 'learning_rate': 3.787388843977365e-06, 'epoch': 1.25}
{'loss': 0.0001, 'learning_rate': 3.7772

KeyboardInterrupt: 

In [ ]:
create_submission(model, f"submission.csv")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
c:\Users\Bernd\anaconda3\envs\mytorch\lib\site-packages\transformers\convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


AttributeError: 'TokenClassifierOutput' object has no attribute 'argmax'